In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
import os
from skimage import io, transform
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchsummary import summary
import pandas as pd
import time
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from emb_descricoes import preprocess

import argparse


parser = argparse.ArgumentParser(description='Arguments for VAER')


parser.add_argument('EMB_DIM', action="store", type=int)
parser.add_argument('DEC_HID_DIM', action="store", type=int)
parser.add_argument('ENC_HID_DIM', action="store", type=int)
parser.add_argument('ATTN_DIM', action="store", type=int)

parser.add_argument('DROPOUT', action="store", type=float)
parser.add_argument('LR', action="store", type=float)

print(parser.parse_args())
read = parser.parse_args()

ENC_HID_DIM = read.ENC_HID_DIM
ENC_EMB_DIM = read.EMB_DIM
DEC_EMB_DIM = read.EMB_DIM
DEC_HID_DIM = read.DEC_HID_DIM



ATTN_DIM = read.ATTN_DIM
ENC_DROPOUT = read.DROPOUT
DEC_DROPOUT = read.DROPOUT
LR = read.LR

EMBEDDING_DIM = 300
LOG_INTERVAL = 40 
MAX_LEN = 200
BATCH_SIZE = 16
N_EPOCHS = 20

Using TensorFlow backend.
usage: ipykernel_launcher.py [-h] EMB_DIM ENC_HID_DIM DEC_HID_DIM  DROPOUT  LR
ipykernel_launcher.py: error: argument EMB_DIM: invalid int value: '/home/danielprado/.local/share/jupyter/runtime/kernel-f85cc4eb-4dc2-4a30-bbdf-04d477db0c2b.json'


SystemExit: 2

/home/danielprado/miniconda3/envs/portinari_pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
embedding_matrix, portinari_idx, retratos_idx, resto_idx, word_index, index_word = preprocess(True)
no_cuda = False
seed = 1
cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
torch.manual_seed(seed)

class PortinariDesc(Dataset):
    def __init__(self, data) -> None:
        super().__init__()
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return len(self.data)
    
#cria o dataset


Achou 526 retratos


/home/danielprado/miniconda3/envs/portinari_pytorch/lib/python3.7/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


total words 571096
null words 2376
0.42 % de palavras não encontradas no emb


In [3]:
from sklearn import model_selection
train_retratos, val_retratos = model_selection.train_test_split(retratos_idx, test_size = 0.2, shuffle = True)
train_resto, val_resto = model_selection.train_test_split(resto_idx, test_size = 0.2, shuffle = True)


train = train_retratos + train_resto
val = val_retratos + val_resto

In [4]:

train = [sent[:MAX_LEN] for sent in train]
val = [sent[:MAX_LEN] for sent in val]

In [5]:
train_tensor = list(map(lambda x: torch.tensor(x), train))
val_tensor = list(map(lambda x: torch.tensor(x), val))

train_dataset = PortinariDesc(train_tensor)
val_dataset = PortinariDesc(val_tensor)

def pad_collate(batch):
    xx = batch
    x_lens = list(map(len, xx))

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0, )

    return xx_pad, x_lens

#embedding_torch = nn.Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1])

In [6]:
'''unknown_tokens = [i for i in range(len(embedding_matrix)) if np.sum(embedding_matrix[i]-embedding_matrix[-1]) == 0]
for tk in unknown_tokens:
    index_word[tk] = '<unk>'
for w, idx in word_index.items():
    if np.sum(embedding_matrix[idx])==0:
        word_index[w] = len(embedding_matrix)-1
palavras_desconhecidas = [w for w, idx in word_index.items()  if np.sum(embedding_matrix[idx]-embedding_matrix[-1]) == 0]
len(palavras_desconhecidas)'''

"unknown_tokens = [i for i in range(len(embedding_matrix)) if np.sum(embedding_matrix[i]-embedding_matrix[-1]) == 0]\nfor tk in unknown_tokens:\n    index_word[tk] = '<unk>'\nfor w, idx in word_index.items():\n    if np.sum(embedding_matrix[idx])==0:\n        word_index[w] = len(embedding_matrix)-1\npalavras_desconhecidas = [w for w, idx in word_index.items()  if np.sum(embedding_matrix[idx]-embedding_matrix[-1]) == 0]\nlen(palavras_desconhecidas)"

In [7]:
embedding_tensor = torch.Tensor(embedding_matrix).to(device)


train_data_loader = torch.utils.data.DataLoader(
    train_dataset, drop_last=True,
    batch_size = BATCH_SIZE, shuffle=True, collate_fn=pad_collate)

val_data_loader = torch.utils.data.DataLoader(
    val_dataset, drop_last=True,
    batch_size = BATCH_SIZE, shuffle=True, collate_fn=pad_collate)

In [8]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

from text_models import Encoder, Decoder, Attention, Seq2Seq

In [9]:
'''ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ATTN_DIM = 64
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5'''



enc = Encoder(embedding_tensor, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT).to(device)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM).to(device)
dec = Decoder(embedding_tensor, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn).to(device)
model = Seq2Seq(enc, dec, device).to(device)

In [10]:

def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)
print("")

In [11]:

optimizer = optim.Adam(model.parameters(), lr=LR)

In [12]:

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 4,429,283 trainable parameters


In [13]:
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [14]:
'''import math
import time

(x,x_l) = next(iter(data_loader))
x = x.to(device)
x_l = torch.Tensor(x_l).to(device)
output = model(x.permute(1,0),x_l)'''

'import math\nimport time\n\n(x,x_l) = next(iter(data_loader))\nx = x.to(device)\nx_l = torch.Tensor(x_l).to(device)\noutput = model(x.permute(1,0),x_l)'

In [15]:
len(train_data_loader)

226

In [16]:
import math
import time

def train(model: nn.Module,
          iterator: DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float,
          epoch):
    
    epoch_loss = 0
    epoch_start = 0
    train_loss = 0
    start = time.time()
    
    model.train(True)
    
    for batch_idx, (x,x_l) in enumerate(iterator):
        
        x = x.to(device)
       
        x_l = torch.Tensor(x_l).to(device)
        
        #print(f"x = {x.shape}, x_l = {x_l.shape}")
        
        output = model(x.permute(1,0),x_l)
        
        optimizer.zero_grad()
        
        flat_output = output.view(-1, output.shape[-1])
        target = x.permute(1,0).contiguous().view(-1)
        
        loss = criterion(flat_output, target)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

        
        if batch_idx % LOG_INTERVAL == 0:
                interval = time.time() - start
                start = time.time()
                epoch_start = epoch_start + interval

                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss : {:.6f} \tTime Interv: {:.6f}'.format(
                    epoch, batch_idx * len(x), len(iterator.dataset),
                           100. * batch_idx / len(iterator),
                           loss.item(), interval))
        #del(x)
        #del(x_l)
        #torch.cuda.empty_cache()
            
    return epoch_loss / len(iterator)

In [17]:
def evaluate(model: nn.Module,
             iterator: DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for batch_idx, (x,x_l) in enumerate(iterator):

            x = x.to(device)

            x_l = torch.Tensor(x_l).to(device)
            
            output = model(x.permute(1,0),x_l, 0)

            flat_output = output.view(-1, output.shape[-1])
            target = x.permute(1,0).contiguous().view(-1)

            loss = criterion(flat_output, target)

            epoch_loss += loss.item()
            del(x)
            del(x_l)
            torch.cuda.empty_cache()

    return epoch_loss / len(iterator)

In [18]:
def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


CLIP = 1

best_valid_loss = float('inf')
last_valid_loss = 100

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss = train(model, train_data_loader, optimizer, criterion, CLIP, epoch)
    valid_loss = evaluate(model, val_data_loader, criterion)
    if valid_loss > last_valid_loss:
        print('\nshould break!!!')
        epoch = N_EPOCHS;
    last_valid_loss = valid_loss
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Train Epoch: 0 [0/3630 (0%)]	Loss : 9.046326 	Time Interv: 0.980478
Train Epoch: 0 [640/3630 (18%)]	Loss : 5.897495 	Time Interv: 31.417345
Train Epoch: 0 [1280/3630 (35%)]	Loss : 5.913294 	Time Interv: 31.709690
Train Epoch: 0 [1920/3630 (53%)]	Loss : 5.704523 	Time Interv: 31.503156
Train Epoch: 0 [2560/3630 (71%)]	Loss : 5.871801 	Time Interv: 32.381515
Train Epoch: 0 [3200/3630 (88%)]	Loss : 5.647846 	Time Interv: 31.979170
Epoch: 01 | Time: 3m 5s
	Train Loss: 5.958 | Train PPL: 386.956
	 Val. Loss: 5.780 |  Val. PPL: 323.726
Train Epoch: 1 [0/3630 (0%)]	Loss : 5.581927 	Time Interv: 0.690814
Train Epoch: 1 [640/3630 (18%)]	Loss : 5.535550 	Time Interv: 32.007462


KeyboardInterrupt: 

In [39]:
model.train()
(x,x_l) = next(iter(train_data_loader))
x = x.to(device)
x_l_tens = torch.Tensor(x_l).to(device)
output = model(x.permute(1,0),x_l_tens, 0)
out = output.argmax(-1).transpose(1,0).cpu().numpy().tolist()
inp = x.cpu().numpy().tolist()
inp_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
          for sentence in inp]
out_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
              for sentence in out]

print('\nTrain = True')
print('\nfrase inp:', inp_text[0][0:x_l[0]+1])
print('\nfrase out:',  out_text[0][0:x_l[0]+1])

print('\nTrain = False')

model.eval()
output = model(x.permute(1,0),x_l_tens, 0)
out = output.argmax(-1).transpose(1,0).cpu().numpy().tolist()
inp = x.cpu().numpy().tolist()
inp_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
          for sentence in inp]
out_text = [[index_word[token] if token != 0 else 'NUll' for token in sentence ] 
              for sentence in out]
print('\nfrase inp:', inp_text[0][0:x_l[0]+1])
print('\nfrase out:',  out_text[0][0:x_l[0]+1])
